In [0]:
import numpy as np 
import pandas as pd 
from pandas import DataFrame 

import matplotlib.pyplot as plt 
import seaborn as sns
import itertools
%matplotlib inline
from matplotlib.pyplot import figure
import matplotlib as mpl
import matplotlib.font_manager as fm
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams["font.family"] = 'Malgun Gothic' 
plt.rcParams["font.size"] = 10
%matplotlib inline 
from tqdm import tqdm_notebook


from datetime import datetime, timedelta 
from statsmodels.tsa.arima_model import ARIMA 

from sklearn.preprocessing import StandardScaler
import os

In [0]:
from pydrive.auth import GoogleAuth

from pydrive.drive import GoogleDrive

from google.colab import auth

from oauth2client.client import GoogleCredentials 


# 1. Authenticate and create the PyDrive client.

auth.authenticate_user()

gauth = GoogleAuth()

gauth.credentials = GoogleCredentials.get_application_default()

drive = GoogleDrive(gauth)

In [0]:
file_id = '1XATewioAnfsVMcZiGN6AYh9ZsMRFX4Yd'

downloaded = drive.CreateFile({'id': file_id})

downloaded.GetContentFile('TEST_DHweightDaconArima_NAFilled.csv')

In [0]:
file_id = '1Kkv55I6cT7DgEFyeC-ryoq2wi3Aq2JSI'

downloaded = drive.CreateFile({'id': file_id})

downloaded.GetContentFile('hourly.csv')

In [0]:
# hourly WEATHER data
w_hr = pd.read_csv("hourly.csv",encoding='euc-kr', sep=',')
w_hr2 = w_hr.loc[:,['Time', 'Temp','Humidity', 'WindSpeed']]
w_hr2['Humidity']=w_hr2.loc[:,'Humidity'].fillna(0)
w_hr2.head(2)

Time  Temp  Humidity  WindSpeed
0  2016.7.20 0:00  25.7       0.0        1.3
1  2016.7.20 1:00  25.6       0.0        1.4

In [0]:
w_hr2['Time'] = pd.to_datetime(w_hr2.Time)
w_hr2 = w_hr2.set_index('Time')
w_hr2.head(2)

Temp  Humidity  WindSpeed
Time                                          
2016-07-20 00:00:00  25.7       0.0        1.3
2016-07-20 01:00:00  25.6       0.0        1.4

In [0]:
test = pd.read_csv("TEST_DHweightDaconArima_NAFilled.csv", index_col=0, sep=',')

In [0]:
test.index = pd.to_datetime(test.index)

In [0]:
def dateTimeVar(df):
    df1 = pd.DataFrame(df.index)
    df1.index = pd.to_datetime(df1.iloc[:,0])
    df1 = df1.drop(columns=df1.columns)
    # datetime을 사용한 파생변수
    # 연도, 월, 일, 요일, 시간
    df1['Mth']= df1.index.month
    df1['Date']=df1.index.day
    df1['Day']=df1.index.weekday
    df1['hour']=df1.index.hour
    
    # Season 1: 봄 ~ 4: 겨울 
    df1['season'] = df1.Mth.apply(lambda x: 4 if x in [12,1,2] else 
                1 if x in [3,4,5] else 2 if x in [6,7,8]
               else 3)
    # dayClass 1: 월, 2: 화~금, 3: 토, 일 
    # 참조 from 실시간 수요예측 기법 개발 및 온라인 수요예측 방안 연구_KPX 한국전력거래소)
    df1['dayClass'] = df1.Day.apply(lambda x: 1 if x ==0 else 
                2 if x in [1,2,3,4] else 3)
    return df1

In [0]:
TE_time_vars = dateTimeVar(test)

In [0]:
submission_hourly = pd.date_range(start = pd.datetime(2018, 7, 1, 0),
                             end = pd.datetime(2018, 7, 1, 23),
                             freq = "H")
tmp_test = pd.DataFrame(index = submission_hourly)

In [0]:
X_test_H = dateTimeVar(tmp_test)

In [0]:
def dummies(df, dum_lst):
    df1 = df.copy()
    # dum_lst에 들어있는 각각의 변수마다 
    # tmp = get_dummies, 해당 original 변수는 drop
    # tmp와 변수가 drop된 df를 join 후 리턴
    for v in dum_lst:
        tmp = pd.get_dummies(df1[v], prefix=v)
        df1 = df1.drop(columns=v).join(tmp)
    return df1

In [0]:
Mer = pd.concat([TE_time_vars, X_test_H])
Mer1 = dummies(Mer, ['season', 'dayClass'])
TE_time_vars1 = Mer1.iloc[:7334].copy()
X_test_H1 = Mer1.iloc[7334:7358].copy()

In [0]:
X_test_H = pd.concat([X_test_H1, w_hr2], axis=1,join='inner')

In [0]:
TE_timeV_dum = TE_time_vars1.copy()

TE_timeV_dum_wthr= pd.concat([TE_timeV_dum, w_hr2], axis=1,join='inner')

In [0]:
TE_timeV_dum

Mth  Date  Day  ...  dayClass_1  dayClass_2  dayClass_3
2017-08-29 10:00:00    8    29    1  ...           0           1           0
2017-08-29 11:00:00    8    29    1  ...           0           1           0
2017-08-29 12:00:00    8    29    1  ...           0           1           0
2017-08-29 13:00:00    8    29    1  ...           0           1           0
2017-08-29 14:00:00    8    29    1  ...           0           1           0
...                  ...   ...  ...  ...         ...         ...         ...
2018-06-30 19:00:00    6    30    5  ...           0           0           1
2018-06-30 20:00:00    6    30    5  ...           0           0           1
2018-06-30 21:00:00    6    30    5  ...           0           0           1
2018-06-30 22:00:00    6    30    5  ...           0           0           1
2018-06-30 23:00:00    6    30    5  ...           0           0           1

[7334 rows x 11 columns]

In [0]:
X_test_H.head()

Mth  Date  Day  ...  Temp  Humidity  WindSpeed
2018-07-01 00:00:00    7     1    6  ...  23.7       0.0        1.7
2018-07-01 01:00:00    7     1    6  ...  23.7       0.0        2.0
2018-07-01 02:00:00    7     1    6  ...  23.8       0.0        2.0
2018-07-01 03:00:00    7     1    6  ...  23.8       0.0        0.7
2018-07-01 04:00:00    7     1    6  ...  23.7       0.0        2.9

[5 rows x 14 columns]

In [0]:
import datetime

time = []

for i in range(3672):
    timeStr = '2018-07-01 00:00:00'
    start = datetime.datetime.strptime(timeStr, '%Y-%m-%d %H:%M:%S')
    new_time = start + datetime.timedelta(hours=i)
    time.append(new_time)
    print(time)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
time1 = pd.DataFrame(time)
time1 = time1.set_index(0)

In [0]:
time1.index[0]

Timestamp('2018-07-01 00:00:00')

In [0]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

lg_param = {"eta" : [0.2],
           "Min_child_weight" : [1],
           "Max_depth" : [3]}

def gsLGBR_Hrly_FitPred (model, params, mID, X_train, y_train, X_test):
    grid_gb = GridSearchCV(model, params, cv=4, verbose=5)
    grid_gb.fit(X_train, y_train)
    fcst = grid_gb.predict(X_test)
    a = pd.DataFrame([mID], columns=['meter_id'])
    for i in range(3672):
        a[time1.index[i]]=[fcst[i]]
    return a
   
def test50_Hrly_GS_LGBReg(train, X_test_H):
    hrly_50 = pd.DataFrame()
    for mID in tqdm_notebook(train.columns):
        data = pd.concat([train.loc[:,[mID]], TE_timeV_dum_wthr], axis=1)
        data = data.rename(columns={mID:'y'})
        #for i in [24, 168]:
        #    data['lag_{}'.format(i)] = data.y.shift(i)
        y_train = data.dropna().y
        X_train = data.dropna().drop(['y'], axis=1)
        
        #X_test_H['lag_24'] = y_train.tail(24).values
        #X_test_H['lag_168'] = data.loc[:,'lag_168'][7166:7190].values
        
        model = xgb.XGBRegressor(n_estimators=1000)
        tmp = gsLGBR_Hrly_FitPred(model, lg_param,
                        mID, X_train, y_train, X_test_H)
        hrly_50 = pd.concat([hrly_50, tmp], axis=0)
    return hrly_50

In [0]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

lg_param = {"eta" : [0.2],
           "Min_child_weight" : [1],
           "Max_depth" : [3]}

def gsLGBR_Hrly_FitPred (model, params, mID, X_train, y_train, X_test):
    grid_gb = GridSearchCV(model, params, cv=4, verbose=5)
    grid_gb.fit(X_train, y_train)
    fcst = grid_gb.predict(X_test)
    a = pd.DataFrame([mID], columns=['meter_id'])
    for i in range(24):
        a['X2018_7_1_'+str(i+1)+'h']=[fcst[i]]
    return a
   
def test50_Hrly_GS_LGBReg(train, X_test_H):
    hrly_50 = pd.DataFrame()
    for mID in tqdm_notebook(train.columns):
        data = pd.concat([train.loc[:,[mID]], TE_timeV_dum_wthr], axis=1)
        data = data.rename(columns={mID:'y'})
        for i in [24, 168]:
            data['lag_{}'.format(i)] = data.y.shift(i)
        y_train = data.dropna().y
        X_train = data.dropna().drop(['y'], axis=1)
        
        X_test_H['lag_24'] = y_train.tail(24).values
        X_test_H['lag_168'] = data.loc[:,'lag_168'][7166:7190].values
        
        model = xgb.XGBRegressor(n_estimators=1000)
        tmp = gsLGBR_Hrly_FitPred(model, lg_param,
                        mID, X_train, y_train, X_test_H)
        hrly_50 = pd.concat([hrly_50, tmp], axis=0)
    return hrly_50

In [0]:
TE_timeV_dum_wthr

Mth  Date  Day  ...  Temp  Humidity  WindSpeed
2017-08-29 10:00:00    8    29    1  ...  19.3       0.0        5.0
2017-08-29 11:00:00    8    29    1  ...  19.7       0.0        4.7
2017-08-29 12:00:00    8    29    1  ...  19.4       0.0        4.8
2017-08-29 13:00:00    8    29    1  ...  19.8       0.0        4.5
2017-08-29 14:00:00    8    29    1  ...  20.5       0.0        3.4
...                  ...   ...  ...  ...   ...       ...        ...
2018-06-30 19:00:00    6    30    5  ...  26.5       0.0        1.8
2018-06-30 20:00:00    6    30    5  ...  24.5       0.0        2.4
2018-06-30 21:00:00    6    30    5  ...  24.5       0.0        3.6
2018-06-30 22:00:00    6    30    5  ...  24.1       0.0        1.4
2018-06-30 23:00:00    6    30    5  ...  23.9       0.0        0.0

[7334 rows x 14 columns]

In [0]:
test.columns

Index(['X26', 'X303', 'X241', 'X435', 'X402', 'X352', 'X305', 'X350', 'X326',
       'X299',
       ...
       'X283', 'X329', 'X223', 'X266', 'X20', 'X443', 'X347', 'X75', 'X107',
       'X230'],
      dtype='object', length=200)

In [0]:
X_test_H

Mth  Date  Day  ...  Temp  Humidity  WindSpeed
2018-07-01 00:00:00    7     1    6  ...  23.7       0.0        1.7
2018-07-01 01:00:00    7     1    6  ...  23.7       0.0        2.0
2018-07-01 02:00:00    7     1    6  ...  23.8       0.0        2.0
2018-07-01 03:00:00    7     1    6  ...  23.8       0.0        0.7
2018-07-01 04:00:00    7     1    6  ...  23.7       0.0        2.9
2018-07-01 05:00:00    7     1    6  ...  23.1       0.3        1.9
2018-07-01 06:00:00    7     1    6  ...  22.2       0.6        2.9
2018-07-01 07:00:00    7     1    6  ...  22.2       0.0        2.6
2018-07-01 08:00:00    7     1    6  ...  22.2       0.3        3.3
2018-07-01 09:00:00    7     1    6  ...  21.5       2.3        3.2
2018-07-01 10:00:00    7     1    6  ...  21.3       6.7        3.3
2018-07-01 11:00:00    7     1    6  ...  21.4       6.3        3.2
2018-07-01 12:00:00    7     1    6  ...  21.2       6.5        5.1
2018-07-01 13:00:00    7     1    6  ...  21.3       4.2        3.8
2018-07-01 14:00:00    7     1    6  ...  21.6       3.3        4.3
2018-07-01 15:00:00    7     1    6  ...  21.9       2.7        3.1
2018-07-01 16:00:00    7     1    6  ...  21.9       2.3        4.4
2018-07-01 17:00:00    7     1    6  ...  21.9       3.8        4.0
2018-07-01 18:00:00    7     1    6  ...  21.8       1.8        3.9
2018-07-01 19:00:00    7     1    6  ...  21.6       1.8        5.1
2018-07-01 20:00:00    7     1    6  ...  21.4       1.3        5.0
2018-07-01 21:00:00    7     1    6  ...  21.3       4.2        4.5
2018-07-01 22:00:00    7     1    6  ...  21.3       6.2        4.8
2018-07-01 23:00:00    7     1    6  ...  21.2       4.6        4.1

[24 rows x 14 columns]

In [0]:
pred_2 = test50_Hrly_GS_LGBReg(test, X_test_H)

Fitting 4 folds for each of 1 candidates, totalling 4 fits
[CV] Max_depth=3, Min_child_weight=1, eta=0.2 ........................
[07:40:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[CV]  Max_depth=3, Min_child_weight=1, eta=0.2, score=-7.745, total=   2.7s
[CV] Max_depth=3, Min_child_weight=1, eta=0.2 ........................
[07:40:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.7s remaining:    0.0s
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[CV]  Max_depth=3, Min_child_weight=1, eta=0.2, score=-0.001, total=   2.6s
[CV] Max_depth=3, Min_child_weight=1, eta=0.2 ........................
[07:40:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

[CV]  Max_depth=3, Min_child_weight=1, eta=0.2, score=-4.006, total=   2.6s
[CV] Max_depth=3, Min_child_weight=1, eta=0.2 ........................
[07:40:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    8.0s remaining:    0.0s
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[CV]  Max_depth=3, Min_child_weight=1, eta=0.2, score=0.039, total=   1.4s
[07:40:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    9.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    9.4s finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


IndexError: ignored

In [0]:
pred_2

meter_id  X2018_7_1_1h  ...  X2018_7_1_23h  X2018_7_1_24h
0       X26      1.121822  ...       1.397714       1.353700
0      X303      0.728295  ...       0.745418       0.660964
0      X241      0.178877  ...       0.290481       0.233181
0      X435      0.302481  ...       0.574616       0.526973
0      X402      0.413689  ...       0.895658       0.792225
..      ...           ...  ...            ...            ...
0      X443      0.329918  ...       0.430530       0.412385
0      X347      0.098193  ...       0.442241       0.335843
0       X75      0.533818  ...       2.667849       1.537334
0      X107      0.097933  ...      -0.030460       0.007814
0      X230      0.433669  ...       0.463238       0.426600

[200 rows x 25 columns]

In [0]:
pred_2.to_csv("pred_hour.csv")

In [0]:
!ls -lh 

total 16M
-rw-r--r-- 1 root root 2.6K Oct 31 02:26 adc.json
-rw-r--r-- 1 root root 692K Oct 31 02:31 hourly.csv
-rw-r--r-- 1 root root  93K Oct 31 04:27 pred_hour.csv
drwxr-xr-x 1 root root 4.0K Oct 25 16:58 sample_data
-rw-r--r-- 1 root root  16M Oct 31 02:27 TEST_DHweightDaconArima_NAFilled.csv


In [0]:
from google.colab import files
files.download('pred_hour.csv')